In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('firsttree').master('local[4]').getOrCreate()

22/06/14 10:43:25 WARN Utils: Your hostname, mehrdad-Standard resolves to a loopback address: 127.0.1.1; using 128.179.176.230 instead (on interface wlp4s0)
22/06/14 10:43:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 10:43:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 10:43:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/14 10:43:26 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/14 10:43:26 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/06/14 10:43:26 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/06/

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import (
    RandomForestClassifier,
    GBTClassifier,
    DecisionTreeClassifier)

### Load Data

In [4]:
data = spark.read.format('libsvm').load('../data/sample_libsvm_data.txt')

22/06/14 10:43:27 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
data.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [6]:
data.select('features').take(1)

[Row(features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0, 356: 253.

### Test Train Split

In [7]:
train_data, test_data = data.randomSplit([0.8, 0.2])

### Three Default Trees

In [8]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [9]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [10]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [11]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[122,123,124...|   [35.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [35.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [35.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [35.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[129,130,131...|   [35.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [35.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [35.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[181,182,183...|   [35.0,0.0]|  [1.0,0.0]|       0.0|
|  1.0|(692,[124,125,126...|   [0.0,47.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[127,128,129...|   [0.0,47.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[127,128,155...|   [0.0,47.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[129,130,131...|   [0.0,47.0]|  [0.0,1.0]|       1.0|
|  1.0|(69

### Evaluate

In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [13]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [14]:
print('DT Accuracy', acc_eval.evaluate(dtc_preds))
print('RFC Accuracy', acc_eval.evaluate(rfc_preds))
print('GBT Accuracy', acc_eval.evaluate(gbt_preds))

DT Accuracy 1.0
RFC Accuracy 1.0
GBT Accuracy 1.0


### Analyzing Feature Importance

In [15]:
rfc_model.featureImportances

SparseVector(692, {99: 0.0005, 100: 0.0006, 101: 0.001, 126: 0.0004, 131: 0.0006, 179: 0.0017, 181: 0.0009, 190: 0.001, 205: 0.0004, 206: 0.0005, 209: 0.0004, 212: 0.0007, 213: 0.0004, 215: 0.0016, 235: 0.0004, 240: 0.0007, 241: 0.0015, 243: 0.0004, 244: 0.0071, 261: 0.0009, 263: 0.0002, 266: 0.0007, 268: 0.0005, 271: 0.0145, 272: 0.0014, 287: 0.0035, 289: 0.0027, 290: 0.007, 291: 0.0031, 295: 0.0003, 296: 0.0004, 299: 0.0007, 300: 0.0098, 313: 0.0008, 317: 0.0086, 318: 0.0009, 322: 0.0058, 323: 0.0004, 326: 0.0002, 327: 0.0005, 328: 0.0145, 329: 0.01, 330: 0.0027, 331: 0.0027, 332: 0.0028, 341: 0.0026, 346: 0.0003, 349: 0.001, 350: 0.0181, 351: 0.0191, 352: 0.0057, 356: 0.0154, 357: 0.0068, 359: 0.0062, 369: 0.0014, 370: 0.0004, 372: 0.0004, 374: 0.001, 375: 0.0004, 378: 0.0282, 379: 0.0141, 380: 0.0006, 384: 0.0013, 385: 0.007, 388: 0.0006, 399: 0.008, 400: 0.0237, 401: 0.0094, 402: 0.0017, 405: 0.0404, 406: 0.0796, 407: 0.0506, 408: 0.0015, 409: 0.001, 410: 0.0006, 411: 0.0025, 412: